In [1]:
import cv2
import numpy as np
import os
import random

cap    = cv2.VideoCapture(0)
print('height:{} width:{}'.format(int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))))

lowerT = 8000
higherT = 20000

fgbg = cv2.createBackgroundSubtractorMOG2()
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
pizza = cv2.imread('pizza.png')
pizza = cv2.resize(pizza, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 3, int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 4))
bridge = cv2.imread('bridge.jpg')
bridge = cv2.resize(bridge, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 6))
fakeEye = cv2.imread('fakeEye.png')


def droppingBridge(frame, i, facesPos):
    top = i
    bottom = bridge.shape[0] + i
    hit = False
    faceBottom = facesPos[0][0] + facesPos[0][1]
    if not(facesPos[0][0] > bottom or faceBottom < top):
        hit = True
    if bottom < frame.shape[0]:
        frame[top:bottom] = bridge
    return hit


def droppingPizza(frame, i, col, h, facesPos):
    top = i
    bottom = h + i
    left = pizza.shape[1] * col
    right = pizza.shape[1] * (col+1)
    hit = False
    mouth1X = facesPos[0][2]+facesPos[0][3]//2
    mouth1Y = facesPos[0][0]+facesPos[0][1] * 4 // 5
    mouth2X = facesPos[1][2]+facesPos[1][3]//2
    mouth2Y = facesPos[1][0]+facesPos[1][1] * 4 // 5
    if (mouth1X < right and mouth1X > left and mouth1Y < bottom and mouth1Y > top):
        hit = True
    if (mouth2X < right and mouth2X > left and mouth2Y < bottom and mouth2Y > top):
        hit = True
    if bottom < frame.shape[0]:
        frame[top:bottom, left:right] = pizza[:h]
    return hit
    

def calMovingAreaRate(img): ##傳入backgroundsubtractor的圖片，計算前景面積
    mvArea = np.count_nonzero(img == 255)
    if mvArea > higherT:
        return 1 #移動面積超過高門檻判定為移動
    elif mvArea < lowerT:
        return 0 #移動面積低過低門檻判定為隱形
    return (mvArea - lowerT) / higherT  ##
    

def detectFace(frame):
    """ Input = frame from video stream
        Output = Image with rectangle box in the face
    """
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Now get the tuples that detect the faces using above cascade
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # faces are the tuples of 4 numbers
    # x,y => upperleft corner coordinates of face
    # width(w) of rectangle in the face
    # height(h) of rectangle in the face
    # grey means the input image to the detector
    # 1.3 is the kernel size or size of image reduced when applying the detection
    # 5 is the number of neighbors after which we accept that is a face
    
    i = 0
    
    face0 = np.zeros(0)
    face1 = np.zeros(0)
    facesPos = np.zeros((10,4))
    eyesPos = np.zeros((10,2,4))
  
    # Now iterate over the faces and detect eyes
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        # Arguements => image, top-left coordinates, bottomright coordinates, color, rectangle border thickness
        
        facesPos[i][0] = y
        facesPos[i][1] = h
        facesPos[i][2] = x
        facesPos[i][3] = w  ##記錄各張臉的位置
        if i==0:
            face0 = frame[y:y+h, x:x+w, :].copy()
        if i==1:
            face1 = frame[y:y+h, x:x+w, :].copy()  ##紀錄兩張臉的樣子，以便之後換臉
        # we now need two region of interests(ROI) grey and color for eyes one to detect and another to draw rectangle
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        # Detect eyes now
        eyes = eyes_cascade.detectMultiScale(roi_gray, 1.2, 3)
        j = 0
        for (ex, ey, ew, eh) in eyes:
            if j > 1: #一次只有一雙眼
                break
            eyesPos[i][j][0] = ey + y
            eyesPos[i][j][1] = eh
            eyesPos[i][j][2] = ex + x
            eyesPos[i][j][3] = ew ##記錄每個眼睛的位置
            j+=1            
        i+=1
        
    facesPos = np.array(facesPos,dtype=np.int)
    eyesPos = np.array(eyesPos, dtype=np.int)  ##必須是int型態才能resize
    if(face1.ndim == 3): ##當出現第二張臉時，換臉
        face1 = cv2.resize(face1, (int(facesPos[0][1]), int(facesPos[0][3])))
        face0 = cv2.resize(face0, (int(facesPos[1][1]), int(facesPos[1][3])))
        frame[facesPos[0][0]:facesPos[0][0]+facesPos[0][1], facesPos[0][2]:facesPos[0][2]+facesPos[0][3]] = face1
        frame[facesPos[1][0]:facesPos[1][0]+facesPos[1][1], facesPos[1][2]:facesPos[1][2]+facesPos[1][3]] = face0

    return facesPos, eyesPos

def loadBackground(cap): ##讀取背景
    loadBackgroundCount = 0
    ret, frame = cap.read()
    if ret==False:
        return frame
    while loadBackgroundCount < 20: ##需要20次畫面中都沒有動作才能被判斷為背景
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1) ##畫面左右翻轉
        if ret==False:
            return frame
        gbg = fgbg.apply(frame)
        mvAreaRate = calMovingAreaRate(gbg) #計算前景移動率
        if(mvAreaRate > lowerT):
            loadBackgroundCount = 0
        else:
            loadBackgroundCount+=1
        cv2.imshow('cam', gbg)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:
            return frame
    return frame

background = loadBackground(cap) ##讀取背景

pizzaPos = 0
bridgePos = 0
speed = 18
score = 0
pizzaH = pizza.shape[0]
isBridgeDropping = False
    

while True:    
    ret, frame = cap.read()    
    frame = cv2.flip(frame, 1) ##水平翻轉畫面
    
    if(pizzaPos % 10 ==0):
        col = random.randint(0,2)  ##控制pizza在畫面水平隨機移動
    if(pizzaPos == 0):
        pizzaH = pizza.shape[0]    ##讓pizza回歸原本大小
            
    gbg = fgbg.apply(frame)    ##backgroundsubtractor
    mvAreaRate = calMovingAreaRate(gbg) ##計算正在移動的面積
    
    facesPos, eyesPos = detectFace(frame) ##偵測臉、眼位置並回傳    
    backgroundT = background.copy()
    
    ##使用宇智波瞳力
    if(mvAreaRate < 1): #當人的移動率小於一時，開啟寫輪眼
        for es in eyesPos:
            for e in es:
                if(e[1] != 0 or e[3] != 0):
                    fakeEyeT = cv2.resize(fakeEye.copy(), (e[1], e[3]))
                    #cv2.rectangle(backgroundT, (e[2],e[0]), (e[2]+e[3], e[0]+e[1]), (0, 50, 255), -1)
                    backgroundT[e[0]:e[0]+e[1], e[2]:e[2]+e[3]] = fakeEyeT
    overlapping = cv2.addWeighted(backgroundT, 1-mvAreaRate, frame, mvAreaRate, 0)
    
    eat = droppingPizza(overlapping, pizzaPos, col, pizzaH, facesPos)
    if(eat):
        score += speed
        pizzaH -= speed
    
    
    if(score % (speed * 6 * 3) == 0 and score != 0):
        isBridgeDropping = True
    if(bridgePos + bridge.shape[0] == frame.shape[0]):
        bridgePos = 0
        isBridgeDropping = False
    if(isBridgeDropping):
        hit = droppingBridge(overlapping, bridgePos, facesPos)
        if(hit and mvAreaRate == 1):
            cv2.putText(overlapping, "Game Over",(0, frame.shape[1] // 2 - 40), cv2.FONT_HERSHEY_SIMPLEX, 3.65, (0, 0, 255), 10, cv2.LINE_AA)
            cv2.putText(overlapping, "Score: %d"%score,(160, frame.shape[1] // 2 + 40), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5, cv2.LINE_AA)
            cv2.imshow('cam', overlapping)
            cv2.waitKey()
            break ##End
        bridgePos+=speed // 6
        
        
    cv2.putText(overlapping, str(score), (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('cam', overlapping)
    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    pizzaPos+=speed // 3
    pizzaPos = pizzaPos % (frame.shape[0] - pizza.shape[0])
    if(pizzaH <= 0):
        pizzaPos = 0
    
cap.release()
cv2.destroyAllWindows()

height:480 width:640


KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np
import os
import random

cap    = cv2.VideoCapture(0)
print('height:{} width:{}'.format(int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))))

lowerT = 8000
higherT = 20000

fgbg = cv2.createBackgroundSubtractorMOG2()
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
pizza = cv2.imread('pizza.png')
pizza = cv2.resize(pizza, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 3, int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 4))
bridge = cv2.imread('bridge.jpg')
bridge = cv2.resize(bridge, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 4))
fakeEye = cv2.imread('fakeEye.png')


def droppingBridge(frame, i, facesPos):
    top = i
    bottom = bridge.shape[0] + i
    hit = False
    faceBottom = facesPos[0][0] + facesPos[0][1]
    if not(facesPos[0][0] > bottom or faceBottom < top): ##人臉碰到橋就算
        hit = True
    if bottom < frame.shape[0]:
        frame[top:bottom] = bridge
    return hit


def droppingPizza(frame, i, col, h, facesPos):
    top = i
    bottom = h + i
    left = pizza.shape[1] * col
    right = pizza.shape[1] * (col+1)
    hit = False
    mouth1X = facesPos[0][2]+facesPos[0][3]//2
    mouth1Y = facesPos[0][0]+facesPos[0][1] * 4 // 5
    mouth2X = facesPos[1][2]+facesPos[1][3]//2
    mouth2Y = facesPos[1][0]+facesPos[1][1] * 4 // 5
    if (mouth1X < right and mouth1X > left and mouth1Y < bottom and mouth1Y > top): ##player 1嘴巴碰到pizza的範圍就吃到
        hit = True
    if (mouth2X < right and mouth2X > left and mouth2Y < bottom and mouth2Y > top): ##player 2嘴巴碰到pizza的範圍就吃到
        hit = True
    if bottom < frame.shape[0]:
        frame[top:bottom, left:right] = pizza[:h]
    return hit
    

def calMovingAreaRate(img): ##傳入backgroundsubtractor的圖片，計算前景面積
    mvArea = np.count_nonzero(img == 255)
    if mvArea > higherT:
        return 1 #移動面積超過高門檻判定為移動
    elif mvArea < lowerT:
        return 0 #移動面積低過低門檻判定為隱形
    return (mvArea - lowerT) / higherT
    

def detectFace(frame):
    """ Input = frame from video stream
        Output = Image with rectangle box in the face
    """
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Now get the tuples that detect the faces using above cascade
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # faces are the tuples of 4 numbers
    # x,y => upperleft corner coordinates of face
    # width(w) of rectangle in the face
    # height(h) of rectangle in the face
    # grey means the input image to the detector
    # 1.3 is the kernel size or size of image reduced when applying the detection
    # 5 is the number of neighbors after which we accept that is a face
    
    i = 0
    
    face0 = np.zeros(0)
    face1 = np.zeros(0)
    facesPos = np.zeros((10,4))
    eyesPos = np.zeros((10,2,4))
  
    # Now iterate over the faces and detect eyes
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        # Arguements => image, top-left coordinates, bottomright coordinates, color, rectangle border thickness
        
        facesPos[i][0] = y
        facesPos[i][1] = h
        facesPos[i][2] = x
        facesPos[i][3] = w  ##記錄各張臉的位置
        if i==0:
            face0 = frame[y:y+h, x:x+w, :].copy()
        if i==1:
            face1 = frame[y:y+h, x:x+w, :].copy()  ##紀錄兩張臉的樣子，以便之後換臉
        # we now need two region of interests(ROI) grey and color for eyes one to detect and another to draw rectangle
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        # Detect eyes now
        eyes = eyes_cascade.detectMultiScale(roi_gray, 1.2, 3)
        j = 0
        for (ex, ey, ew, eh) in eyes:
            if j > 1: #一次只有一雙眼
                break
            eyesPos[i][j][0] = ey + y
            eyesPos[i][j][1] = eh
            eyesPos[i][j][2] = ex + x
            eyesPos[i][j][3] = ew ##記錄每個眼睛的位置
            j+=1            
        i+=1
        
    facesPos = np.array(facesPos,dtype=np.int)
    eyesPos = np.array(eyesPos, dtype=np.int)  ##必須是int型態才能resize
    if(face1.ndim == 3): ##當出現第二張臉時，換臉
        face1 = cv2.resize(face1, (int(facesPos[0][1]), int(facesPos[0][3])))
        face0 = cv2.resize(face0, (int(facesPos[1][1]), int(facesPos[1][3])))
        frame[facesPos[0][0]:facesPos[0][0]+facesPos[0][1], facesPos[0][2]:facesPos[0][2]+facesPos[0][3]] = face1
        frame[facesPos[1][0]:facesPos[1][0]+facesPos[1][1], facesPos[1][2]:facesPos[1][2]+facesPos[1][3]] = face0

    return facesPos, eyesPos

def loadBackground(cap): ##讀取背景
    loadBackgroundCount = 0
    ret, frame = cap.read()
    if ret==False:
        return frame
    while loadBackgroundCount < 20: ##需要20次畫面中都沒有動作才能被判斷為背景
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        if ret==False:
            return frame
        gbg = fgbg.apply(frame)
        mvAreaRate = calMovingAreaRate(gbg)
        if(mvAreaRate > lowerT):  ##如果有動作就在數一次
            loadBackgroundCount = 0
        else:
            loadBackgroundCount+=1
        cv2.imshow('cam', gbg)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:
            return frame
    return frame

background = loadBackground(cap) ##讀取背景

pizzaPos = 0
bridgePos = 0
speed = 18
score = 0
pizzaH = pizza.shape[0]
isBridgeDropping = False
    

while True:    
    ret, frame = cap.read()    
    frame = cv2.flip(frame, 1) ##水平翻轉畫面
    
    if(pizzaPos % 10 ==0):
        col = random.randint(0,2)  ##控制pizza在畫面水平隨機移動
    if(pizzaPos == 0):
        pizzaH = pizza.shape[0]    ##讓pizza回歸原本大小
            
    gbg = fgbg.apply(frame)    ##backgroundsubtractor
    mvAreaRate = calMovingAreaRate(gbg) ##計算正在移動的面積
    
    facesPos, eyesPos = detectFace(frame) ##偵測臉、眼位置並回傳    
    backgroundT = background.copy()
    
    ##使用宇智波瞳力
    if(mvAreaRate < 1): #當人的移動率小於一時，開啟寫輪眼
        for es in eyesPos:
            for e in es:
                if(e[1] != 0 or e[3] != 0):
                    fakeEyeT = cv2.resize(fakeEye.copy(), (e[1], e[3]))
                    #cv2.rectangle(backgroundT, (e[2],e[0]), (e[2]+e[3], e[0]+e[1]), (0, 50, 255), -1)
                    backgroundT[e[0]:e[0]+e[1], e[2]:e[2]+e[3]] = fakeEyeT
    overlapping = cv2.addWeighted(backgroundT, 1-mvAreaRate, frame, mvAreaRate, 0)
    
    eat = droppingPizza(overlapping, pizzaPos, col, pizzaH, facesPos) ##以嘴巴位置判斷是否吃到pizza
    if(eat):
        score += speed
        pizzaH -= speed  #被吃的pizza變小
    
    
    if(score % (speed * 6 * 3) == 0 and score != 0): ##當分數到達門檻，叫出bridge干擾人
        isBridgeDropping = True
    if(bridgePos + bridge.shape[0] == frame.shape[0]):#當bridge的位置到達畫面底部時，需要等到下一次觸發bridge dropping
        bridgePos = 0
        isBridgeDropping = False
    if(isBridgeDropping): #橋下來，並判斷人臉式不是撞到橋
        hit = droppingBridge(overlapping, bridgePos, facesPos)
        if(hit and mvAreaRate == 1): ##如果撞到bridge，且沒有用血輪眼進行虛化就 game over
            cv2.putText(overlapping, "Game Over",(0, frame.shape[1] // 2 - 40), cv2.FONT_HERSHEY_SIMPLEX, 3.65, (0, 0, 255), 10, cv2.LINE_AA)
            cv2.putText(overlapping, "Score: %d"%score,(160, frame.shape[1] // 2 + 40), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5, cv2.LINE_AA)
            cv2.imshow('cam', overlapping)
            cv2.waitKey()
            break ##End
        bridgePos+=speed // 6
        
        
    cv2.putText(overlapping, str(score), (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA) #秀分數
    cv2.imshow('cam', overlapping)
    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    pizzaPos+=speed // 3
    pizzaPos = pizzaPos % (frame.shape[0] - pizza.shape[0]) #pizza到底部時回到最上面
    if(pizzaH <= 0): #pizza被吃完時回到最上面
        pizzaPos = 0
    
cap.release()
cv2.destroyAllWindows()

height:480 width:640


KeyboardInterrupt: 